In [1]:
import tensorflow as tf
print(tf.__version__)

import tensorflow_hub as hub

import numpy as np
import os
import pandas as pd
import re

import gzip
import shutil

from tqdm import tqdm
import json
from nltk.tokenize import sent_tokenize

# Reduce logging output.
tf.logging.set_verbosity(tf.logging.ERROR)

1.14.0


In [2]:
def open_file(filename) :
    if filename.split('.')[-1] == "gz" :
        file = gzip.open(filename,'rt')
    else :
        file = open(filename, 'rt')
    return file

def collect_target_samples(filename) :
    
    num_lines = sum(1 for line in open_file(filename))    
    data = open_file(filename)
    
    # collected samples
    target_samples = []
    
    for i, line in tqdm(enumerate(data), total = num_lines) :
    
        jsondata = json.loads(line)
    
        if i == 0 :
            continue
            
        context = jsondata['context']
        context_len = len(context)
        
        # preventing sentence split according to . ! ? in answer texts
        for q in jsondata['qas'] :

            ans =  q['detected_answers'][0]['text']
            pos =  q['detected_answers'][0]['char_spans'][0][0]

            if '.' in ans :
                context = context[:pos] + re.sub('\.', '♬', context[pos : pos + len(ans)]) + context[pos + len(ans):]
                
            if '!' in ans :
                context = context[:pos] + re.sub('!', '♪', context[pos : pos + len(ans)]) + context[pos + len(ans):]
                
            if '?' in ans :
                context = context[:pos] + re.sub('\?', '♩', context[pos : pos + len(ans)]) + context[pos + len(ans):]

        # split passage to sentences
        sen_text = sent_tokenize(context)
        sen_pos = []
        
        sample = dict()
        
        for j in range(len(sen_text)) :

            pos = sum(sen_pos) 
            pos += len(sen_text[j])
                
            # adjusting sentence position because sent_tokenize automatically strips splited sentence texts
            while pos < context_len and ord(context[pos]) in [10, 32, 160] : 
                pos += 1

            sen_pos.append(pos - sum(sen_pos))

            # remove tag texts for improving sentence embedding quality
            sen_text[j] = re.sub('\[TLE\]', ' ', sen_text[j])
            sen_text[j] = re.sub('\[DOC\]', ' ', sen_text[j])
            sen_text[j] = re.sub('\[PAR\]', ' ', sen_text[j])
            
            #sen_text[j] = re.sub('<P>', ' ', sen_text[j])
            #sen_text[j] = re.sub('</P>', ' ', sen_text[j])

            # restore replaced characters
            sen_text[j] = re.sub('♬', '.', sen_text[j])
            sen_text[j] = re.sub('♪', '!', sen_text[j])
            sen_text[j] = re.sub('♩', '?', sen_text[j])

        sample['sentence_text'] = sen_text
        sample['sentence_pos'] = sen_pos
            
        questions = []
        for q in jsondata['qas'] :
            
            # only use first detected answer
            answer = q['detected_answers'][0]
            
            spans = []
            unique_sen = set()
            for j in range(len(answer['char_spans'])) :
                pos = answer['char_spans'][j][0]
                
                # find sentence index including target span
                idx = 0
                for k in range(len(sen_text)) :
                    if pos < (sum(sen_pos[:k+1])) :
                        idx = k
                        break
                
                # ignoring spans with duplicated sentence index
                if idx not in unique_sen :
                    spans.append((answer['char_spans'][j], idx))
                    unique_sen.add(idx)
              
            if len(spans) > 1 :
                questions.append({
                     'qid' : q['qid']
                    ,'question' : q['question']
                    ,'answer'   : answer['text']
                    ,'ans_spans': spans
                })
            
        if len(questions) > 0 :
            sample['question'] = questions
            target_samples.append(sample)

    return target_samples

In [3]:
import pickle

if False :

    train_files = [os.path.join("./MRQA-Shared-Task-2019/download_train", file) for file in os.listdir("./MRQA-Shared-Task-2019/download_train")]
    #dev_files = [os.path.join("./MRQA-Shared-Task-2019/download_in_domain_dev", file) for file in os.listdir("./MRQA-Shared-Task-2019/download_in_domain_dev")]
    #out_dev_files = [os.path.join("./MRQA-Shared-Task-2019/download_out_of_domain_dev", file) for file in os.listdir("./MRQA-Shared-Task-2019/download_out_of_domain_dev")]

    files = train_files
    print(files)
    
    all_target_samples = dict()
    for file in files :
        print(file)
        target_samples = collect_target_samples(file)
        questions = []
        for l in target_samples :
            questions = questions + l['question']
        print("Num. collected samples :", len(questions))
        
        if len(questions) > 0 :
            all_target_samples[file] = target_samples

    with open("all_target_samples_in_domain_train.pickle", 'wb') as handle:
        pickle.dump(all_target_samples, handle)
    
else :
    
    with open("all_target_samples_in_domain_train.pickle", 'rb') as handle:
        all_target_samples = pickle.load(handle)
        
print(all_target_samples.keys())

dict_keys(['./MRQA-Shared-Task-2019/download_train/HotpotQA.jsonl.gz', './MRQA-Shared-Task-2019/download_train/NewsQA.jsonl.gz', './MRQA-Shared-Task-2019/download_train/SQuAD.jsonl.gz', './MRQA-Shared-Task-2019/download_train/TriviaQA.jsonl.gz', './MRQA-Shared-Task-2019/download_train/SearchQA.jsonl.gz'])


In [4]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

def correct_target_samples(data) :
    
    qid = []
    span_num = []
    spans = []
    texts = []
    ans = []
    for d in tqdm(data) :
        sentence_text = d['sentence_text']
        for q in d['question'] :
            texts.append(q['question'])
            spans.append(q['ans_spans'])
            span_texts = [sentence_text[t[1]] for t in q['ans_spans']]
            texts = texts + span_texts      
            qid.append(q['qid'])
            ans.append(q['answer'])
            span_num.append(len(span_texts))
            
    print("Num. quesitons :", len(qid))
    print("Num. texts     :", len(texts))

    # Import the Universal Sentence Encoder's TF Hub module
    module_url = "https://tfhub.dev/google/universal-sentence-encoder/2" #@param ["https://tfhub.dev/google/universal-sentence-encoder/2", "https://tfhub.dev/google/universal-sentence-encoder-large/3"]
    embed = hub.Module(module_url)

    session = tf.Session()
    session.run([tf.global_variables_initializer(), tf.tables_initializer()])
    
    vectorizer = TfidfVectorizer().fit(texts)
    
    correct_spans = []
    batch_size = 1000
    for i in tqdm(range(0, len(qid), batch_size)) :
    
        prev_text_num = sum(span_num[:i]) + i
        next_text_num = sum(span_num[i:i+batch_size]) + batch_size
    
        batch_text = texts[prev_text_num : prev_text_num + next_text_num]

        text_embed = session.run(embed(batch_text))
        text_vect = vectorizer.transform(batch_text)

        for j in range(len(span_num[i:i+batch_size])) :
            
            text_idx = sum(span_num[i:i+j]) + j

            # get sentence embedding similarity
            q_embed = text_embed[text_idx].reshape(1, -1)
            s_embed = text_embed[text_idx+1:text_idx+1+span_num[i+j]].reshape(span_num[i+j], -1)
            embed_sim = cosine_similarity(q_embed, s_embed)

            # get tfidf vector similarity
            q_vect = text_vect[text_idx].reshape(1, -1)
            s_vect = text_vect[text_idx+1:text_idx+1+span_num[i+j]].reshape(span_num[i+j], -1)
            vect_sim = cosine_similarity(q_vect, s_vect)
            
            # blend similarity
            sim = (embed_sim + vect_sim) / 2
        
            most_sim_idx = np.argsort(sim)[0][-1]
            if most_sim_idx != 0 :
                correct_spans.append({
                     "qid" : qid[i+j]
                    ,"question" : texts[text_idx]
                    ,"answer"   : ans[i+j]
                    ,"origin_span" : (texts[text_idx + 1] , spans[i+j][0])
                    ,"revise_span" : (texts[text_idx + 1 + most_sim_idx] , spans[i+j][most_sim_idx])
                })
        
    return correct_spans

"""
for file in all_target_samples.keys() :
    print(file)
    correct_spans = correct_target_samples(all_target_samples[file])
    print("Num. revised spans :", len(correct_spans))
    if len(correct_spans) > 0 :
        with open(file + "_spans.pickle", 'wb') as handle:
            pickle.dump(correct_spans, handle)

all_target_samples.keys()
"""

'\nfor file in all_target_samples.keys() :\n    print(file)\n    correct_spans = correct_target_samples(all_target_samples[file])\n    print("Num. revised spans :", len(correct_spans))\n    if len(correct_spans) > 0 :\n        with open(file + "_spans.pickle", \'wb\') as handle:\n            pickle.dump(correct_spans, handle)\n\nall_target_samples.keys()\n'

In [6]:
for file in all_target_samples.keys() :
    print(file)
    
    try :
        with open(file + "_spans.pickle", 'rb') as handle:
            correct_spans = pickle.load(handle)
    except :
        continue
    
    # make qid index
    qid_index = dict()
    for i, row in enumerate(correct_spans) :
        qid_index[row['qid']] = i
    
    path = '/'.join(file.split('/')[:-1])
    filename = file.split('/')[-1].split('.')[0]
    new_file = open(os.path.join(path, filename + "_revised.jsonl"), "wt")
    
    num_lines = sum(1 for line in open_file(file))    
    data = open_file(file)
    for i, line in tqdm(enumerate(data), total = num_lines) :
    
        jsondata = json.loads(line)
    
        if i == 0 :
            new_file.write(json.dumps(jsondata) + '\n')
            continue

        for j, q in enumerate(jsondata['qas']) :
            
            if q['qid'] in qid_index :
                
                revised_idx = 0
                revised_span = correct_spans[qid_index[q['qid']]]['revise_span'][1][0]
                char_spans = q['detected_answers'][0]['char_spans']
                for k in range(1, len(char_spans)) :
                    if char_spans[k][0] == revised_span[0] and char_spans[k][1] == revised_span[1] :
                        revised_idx = k
                        break
                        
                jsondata['qas'][j]['detected_answers'][0]['token_spans'] = [jsondata['qas'][j]['detected_answers'][0]['token_spans'][revised_idx]]
                
        new_file.write(json.dumps(jsondata) + '\n')
    
    new_file.close()
    
    with open(os.path.join(path, filename + "_revised.jsonl"), 'rt') as f_in:
        with gzip.open(os.path.join(path, filename + "_revised.jsonl") + '.gz', 'wt') as f_out:
            shutil.copyfileobj(f_in, f_out)

./MRQA-Shared-Task-2019/download_train/HotpotQA.jsonl.gz


100%|██████████| 72929/72929 [00:15<00:00, 4800.23it/s]


./MRQA-Shared-Task-2019/download_train/NewsQA.jsonl.gz


100%|██████████| 11429/11429 [00:08<00:00, 1302.17it/s]


./MRQA-Shared-Task-2019/download_train/SQuAD.jsonl.gz


100%|██████████| 18886/18886 [00:04<00:00, 4357.72it/s]


./MRQA-Shared-Task-2019/download_train/TriviaQA.jsonl.gz


100%|██████████| 61689/61689 [01:01<00:00, 1010.49it/s]


./MRQA-Shared-Task-2019/download_train/SearchQA.jsonl.gz


100%|██████████| 117385/117385 [01:49<00:00, 1070.41it/s]


In [5]:
import pandas as pd

data = []
for file in correct_result.keys() :
    for s in correct_result[file] :
        data.append([
             file
            ,s['qid']
            ,s['question']
            ,s['answer']
            ,s['origin_span']
            ,s['revise_span']
        ])

df = pd.DataFrame(data)
df.columns = ['dataset', 'qid', 'question', 'answer', 'origin_span', 'revise_span']
df.to_csv("revise_spans_with_tfidf_out_domain.csv", index = False)

In [13]:
def test(filename) :
    
    num_lines = sum(1 for line in open_file(filename))    
    data = open_file(filename)
    
    # collected samples
    target_samples = []
    
    for i, line in tqdm(enumerate(data), total = num_lines) :
    
        jsondata = json.loads(line)

In [14]:
for file in all_target_samples.keys() :
    path = '/'.join(file.split('/')[:-1])
    filename = file.split('/')[-1].split('.')[0]
    print(filename)
    test(os.path.join(path, filename + "_revised.jsonl.gz"))

HotpotQA


100%|██████████| 72929/72929 [00:06<00:00, 10452.68it/s]


NewsQA


100%|██████████| 11429/11429 [00:05<00:00, 2005.08it/s]


SQuAD


100%|██████████| 18886/18886 [00:01<00:00, 9657.75it/s]


TriviaQA


100%|██████████| 61689/61689 [00:35<00:00, 1746.00it/s]


SearchQA


100%|██████████| 117385/117385 [01:06<00:00, 1761.26it/s]


In [10]:
file1 = '/home/adam/project/mrqa2019_seanie/data/train/SQuAD.jsonl.gz'
file2 = '/home/adam/project/mrqa2019_seanie/data/train_revised/SQuAD.jsonl.gz'
data1 = open_file(file1)
data2 = open_file(file2)

In [11]:
for line1, line2 in zip(data1, data2) :
    #line1 = json.loads(line1)
    #line2 = json.loads(line2)
    #print(line1)
    #print(line2)
    if line1 != line2 :
        line1 = json.loads(line1)
        line2 = json.loads(line2)
        q1 = line1['qas']
        q2 = line2['qas']
        for s1, s2 in zip(q1, q2) :
            if s1['detected_answers'][0]['token_spans'][0] != s2['detected_answers'][0]['token_spans'][0] :
                print(s1)
                print(s2)
                print('')
        print('')

{'answers': ['Forbes'], 'question': 'In 2012 who placed Beyonce at 16 in the Celebrity List?', 'id': '56bfb502a10cfb140055125c', 'qid': '87dff6252a53447f8f8f39970cc0a6cc', 'question_tokens': [['In', 0], ['2012', 3], ['who', 8], ['placed', 12], ['Beyonce', 19], ['at', 27], ['16', 30], ['in', 33], ['the', 36], ['Celebrity', 40], ['List', 50], ['?', 54]], 'detected_answers': [{'text': 'Forbes', 'token_spans': [[0, 0], [293, 293], [122, 122], [86, 86], [337, 337]], 'char_spans': [[0, 5], [1410, 1415], [584, 589], [416, 421], [1611, 1616]]}]}
{'answers': ['Forbes'], 'question': 'In 2012 who placed Beyonce at 16 in the Celebrity List?', 'id': '56bfb502a10cfb140055125c', 'qid': '87dff6252a53447f8f8f39970cc0a6cc', 'question_tokens': [['In', 0], ['2012', 3], ['who', 8], ['placed', 12], ['Beyonce', 19], ['at', 27], ['16', 30], ['in', 33], ['the', 36], ['Celebrity', 40], ['List', 50], ['?', 54]], 'detected_answers': [{'text': 'Forbes', 'token_spans': [[122, 122]], 'char_spans': [[0, 5], [1410, 14

{'answers': ['Columbia'], 'question': 'What company created Epic Records?', 'id': '56df13fdc65bf219000b3f40', 'qid': 'e2151b8cd9a040a0aedd64512ff8493f', 'question_tokens': [['What', 0], ['company', 5], ['created', 13], ['Epic', 21], ['Records', 26], ['?', 33]], 'detected_answers': [{'text': 'Columbia', 'token_spans': [[12, 12], [0, 0], [59, 59]], 'char_spans': [[62, 69], [0, 7], [331, 338]]}]}
{'answers': ['Columbia'], 'question': 'What company created Epic Records?', 'id': '56df13fdc65bf219000b3f40', 'qid': 'e2151b8cd9a040a0aedd64512ff8493f', 'question_tokens': [['What', 0], ['company', 5], ['created', 13], ['Epic', 21], ['Records', 26], ['?', 33]], 'detected_answers': [{'text': 'Columbia', 'token_spans': [[0, 0]], 'char_spans': [[62, 69], [0, 7], [331, 338]]}]}


{'answers': ['Panels'], 'question': 'What are the individual images with action called in comics?', 'id': '56e03c3c7aa994140058e3ed', 'qid': 'f4d8985d000147d5a52b56d65ff3929f', 'question_tokens': [['What', 0], ['are', 5], ['

{'answers': ['The Delhi Metro'], 'question': "What is the name of the world's 12 largest metro system by length?", 'id': '570691b652bb891400689a6c', 'qid': '63a619f736f048399d7b658117787871', 'question_tokens': [['What', 0], ['is', 5], ['the', 8], ['name', 12], ['of', 17], ['the', 20], ['world', 24], ["'s", 29], ['12', 32], ['largest', 35], ['metro', 43], ['system', 49], ['by', 56], ['length', 59], ['?', 65]], 'detected_answers': [{'text': 'The Delhi Metro', 'token_spans': [[1, 2], [30, 31], [45, 46]], 'char_spans': [[4, 14], [150, 160], [223, 233]]}]}
{'answers': ['The Delhi Metro'], 'question': "What is the name of the world's 12 largest metro system by length?", 'id': '570691b652bb891400689a6c', 'qid': '63a619f736f048399d7b658117787871', 'question_tokens': [['What', 0], ['is', 5], ['the', 8], ['name', 12], ['of', 17], ['the', 20], ['world', 24], ["'s", 29], ['12', 32], ['largest', 35], ['metro', 43], ['system', 49], ['by', 56], ['length', 59], ['?', 65]], 'detected_answers': [{'text

{'answers': ['Prince Hall Freemasonry'], 'question': 'What exists today because of the refusal of early American lodges to admit African Americans?', 'id': '57264caef1498d1400e8db70', 'qid': '05d0889327cf4baeaea4c3f4486a0e80', 'question_tokens': [['What', 0], ['exists', 5], ['today', 12], ['because', 18], ['of', 26], ['the', 29], ['refusal', 33], ['of', 41], ['early', 44], ['American', 50], ['lodges', 59], ['to', 66], ['admit', 69], ['African', 75], ['Americans', 83], ['?', 92]], 'detected_answers': [{'text': 'Prince Hall Freemasonry', 'token_spans': [[209, 211], [0, 2]], 'char_spans': [[1057, 1079], [0, 22]]}]}
{'answers': ['Prince Hall Freemasonry'], 'question': 'What exists today because of the refusal of early American lodges to admit African Americans?', 'id': '57264caef1498d1400e8db70', 'qid': '05d0889327cf4baeaea4c3f4486a0e80', 'question_tokens': [['What', 0], ['exists', 5], ['today', 12], ['because', 18], ['of', 26], ['the', 29], ['refusal', 33], ['of', 41], ['early', 44], ['Am

{'answers': ['Chinese political philosophy'], 'question': 'What specifically dates back with Confucius in the 6th century BC?', 'id': '572696dff1498d1400e8e48e', 'qid': '38c104e5856e40b98f778abcf3631eab', 'question_tokens': [['What', 0], ['specifically', 5], ['dates', 18], ['back', 24], ['with', 29], ['Confucius', 34], ['in', 44], ['the', 47], ['6th', 51], ['century', 55], ['BC', 63], ['?', 65]], 'detected_answers': [{'text': 'Chinese political philosophy', 'token_spans': [[21, 23], [0, 2]], 'char_spans': [[124, 151], [0, 27]]}]}
{'answers': ['Chinese political philosophy'], 'question': 'What specifically dates back with Confucius in the 6th century BC?', 'id': '572696dff1498d1400e8e48e', 'qid': '38c104e5856e40b98f778abcf3631eab', 'question_tokens': [['What', 0], ['specifically', 5], ['dates', 18], ['back', 24], ['with', 29], ['Confucius', 34], ['in', 44], ['the', 47], ['6th', 51], ['century', 55], ['BC', 63], ['?', 65]], 'detected_answers': [{'text': 'Chinese political philosophy', 't

{'answers': ['Tibet'], 'question': 'Who managed religious and regional political affairs?', 'id': '5727d1da2ca10214002d974e', 'qid': 'd83948418b9c4b8fb8e77e0263772f84', 'question_tokens': [['Who', 0], ['managed', 4], ['religious', 12], ['and', 22], ['regional', 26], ['political', 35], ['affairs', 45], ['?', 52]], 'detected_answers': [{'text': 'Tibet', 'token_spans': [[60, 60], [0, 0], [75, 75]], 'char_spans': [[360, 364], [0, 4], [441, 445]]}]}
{'answers': ['Tibet'], 'question': 'Who managed religious and regional political affairs?', 'id': '5727d1da2ca10214002d974e', 'qid': 'd83948418b9c4b8fb8e77e0263772f84', 'question_tokens': [['Who', 0], ['managed', 4], ['religious', 12], ['and', 22], ['regional', 26], ['political', 35], ['affairs', 45], ['?', 52]], 'detected_answers': [{'text': 'Tibet', 'token_spans': [[0, 0]], 'char_spans': [[360, 364], [0, 4], [441, 445]]}]}


{'answers': ['Two'], 'question': 'How many ports does Oklahoma have?', 'id': '5727e47d3acd2414000def25', 'qid': '05d29ad

{'answers': ['Wever'], 'question': 'Who said the Luftwaffe General Staff should be taught grand strategy?', 'id': '572f58c5b2c2fd1400568053', 'qid': 'cc051cd40f9c4674bc1cccd85575c13d', 'question_tokens': [['Who', 0], ['said', 4], ['the', 9], ['Luftwaffe', 13], ['General', 23], ['Staff', 31], ['should', 37], ['be', 44], ['taught', 47], ['grand', 54], ['strategy', 60], ['?', 68]], 'detected_answers': [{'text': 'Wever', 'token_spans': [[48, 48], [0, 0]], 'char_spans': [[275, 279], [0, 4]]}]}
{'answers': ['Wever'], 'question': 'Who said the Luftwaffe General Staff should be taught grand strategy?', 'id': '572f58c5b2c2fd1400568053', 'qid': 'cc051cd40f9c4674bc1cccd85575c13d', 'question_tokens': [['Who', 0], ['said', 4], ['the', 9], ['Luftwaffe', 13], ['General', 23], ['Staff', 31], ['should', 37], ['be', 44], ['taught', 47], ['grand', 54], ['strategy', 60], ['?', 68]], 'detected_answers': [{'text': 'Wever', 'token_spans': [[0, 0]], 'char_spans': [[275, 279], [0, 4]]}]}


{'answers': ['Buddhi

{'answers': ['Southern'], 'question': 'What region of italy was part of the Norman empire?', 'id': '57316e2b497a881900248ed3', 'qid': 'e2648f9502dc4e259f29e31450b945a7', 'question_tokens': [['What', 0], ['region', 5], ['of', 12], ['italy', 15], ['was', 21], ['part', 25], ['of', 30], ['the', 33], ['Norman', 37], ['empire', 44], ['?', 50]], 'detected_answers': [{'text': 'Southern', 'token_spans': [[92, 92], [0, 0]], 'char_spans': [[521, 528], [0, 7]]}]}
{'answers': ['Southern'], 'question': 'What region of italy was part of the Norman empire?', 'id': '57316e2b497a881900248ed3', 'qid': 'e2648f9502dc4e259f29e31450b945a7', 'question_tokens': [['What', 0], ['region', 5], ['of', 12], ['italy', 15], ['was', 21], ['part', 25], ['of', 30], ['the', 33], ['Norman', 37], ['empire', 44], ['?', 50]], 'detected_answers': [{'text': 'Southern', 'token_spans': [[0, 0]], 'char_spans': [[521, 528], [0, 7]]}]}


{'answers': ['The Chief Deputy Whip'], 'question': 'Who is the chief vote counter for party?', '